# Final Project IRWA: Twitter Crawling

This section of the code was made in other subject project by Gerard, and adapted to cover the requisites of the data in this project. But it was so prepared to convert the json to csv that was a nonsense not using to old jason from Gerard project and make slight changes to keep only the needed columns in the csv.

## 1. Install and import needed libraries

In [4]:
import tweepy
import simplejson as json
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

import time
import re

## 2.  Gathering tweets in real-time related to US 2020 Elections


Here is the code used to crawl the tweets.

In [5]:
#Twitter Developer keys and tokens
'''
consumer_key=
consumer_secret=
access_token=
access_secret=
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret) '''

In [6]:
# Crawl tweets and store them in 'E:\Tweets\Output_v3.json' and a sample of the dataset in 'E:\Tweets\Output_sample.json'
tweetsPerQry = 100000
maxTweets = 3000000
import json

hashtag = ['Trump OR Biden OR Democrat OR Republican OR Kamala OR Pence OR #Trump OR #Biden OR #Democrat OR #Republican OR #Kamala OR #Pence']
api = tweepy.API(auth)
places = api.geo_search(query="USA", granularity="country")
place_id = places[0].id
place = " -filter:retweets AND place:"+str(place_id)

hashtag = hashtag[0] + place
print(hashtag)
maxId = -1
tweetCount = 0
authentication = tweepy.OAuthHandler(consumer_key, consumer_secret)
authentication.set_access_token(access_token, access_secret)

while tweetCount < maxTweets:



    api = tweepy.API(authentication, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    if(maxId <= 0):
        newTweets = api.search(q=hashtag, count=tweetsPerQry, result_type="recent", tweet_mode="extended")
        
    else:
        newTweets = api.search(q=hashtag, count=tweetsPerQry, max_id=str(maxId - 1), result_type="recent", tweet_mode="extended")
    
    if not newTweets:
        print("Tweet Habis")
        break

    try:
        name='Output_v3.json'
        with open(name, 'a') as f:
            for tweet in newTweets:
                json.dump(tweet._json, f) # This will store the whole JSON data in the file, you can perform some JSON filters
        if(tweetCount==0):
            name='Output_sample.json'
            with open(name, 'a') as f:
                for tweet in newTweets:
                    json.dump(tweet._json+'\n', f)    

            # Setting a limit in the number of tweets collected
            
    except BaseException as e:
        print("Error on_data: %s" % str(e))

    tweetCount += len(newTweets)
    print(tweetCount/maxTweets*100)
    maxId = newTweets[-1].id



Trump OR Biden OR Democrat OR Republican OR Kamala OR Pence OR #Trump OR #Biden OR #Democrat OR #Republican OR #Kamala OR #Pence -filter:retweets AND place:96683cc9126741d1
Error on_data: unsupported operand type(s) for +: 'dict' and 'str'
0.0033333333333333335
0.006666666666666667
0.01
0.013333333333333334
0.016666666666666666
0.02
0.023333333333333334
0.02666666666666667


KeyboardInterrupt: ignored

(The error is beacause we only tried to run for a few iterations and we interrupted)

# 3. Extracting the data

We extract the needed parts of the jsons and transform it into managable file system in csv format.

### Store the JSON data in a CSV for analysing
#### Panel of control


In [ ]:
class control:
    def  __init__(self,location):
        with open(location, 'r', encoding='utf-8') as dat:
            self.data = json.load(dat, encoding='utf-8')
            self.location = location
    def update(self,num):
        self.data["seed"]=str(num)
    def tweet_count(self):
        return(int(self.data["seed"]))
    def window(self):
        return(int(self.data["window"]))
    def start(self):
        return(int(self.data["start"]))
    def end(self):
        return(int(self.data["end"]))
    def save(self):
        with open(self.location, 'w') as dat:
            json.dump(self.data,dat)

In [ ]:
# The file where we saved our tweets
json_file = 'E:\Tweets\Output_v3.json'
json_file_corrected = 'E:\Tweets\Output_v5.json'

# Properties stored in prop.json
properties_data= 'prop.json'

# The file where we are going to save:
# tweet_count,id,created_at,retweet_count,favorite_count,lang,user.id_str,user.screen_name,user.followers_count,
# user.friend_count,user.listed_count,user.created_at,place.full_name,place.country
csv_file_data = 'Output_v5_data.csv'

# This file will store: tweet_count,user.screen_name,entities.user_mentions.screen_name
csv_file_ment = 'Output_v5_ment.csv'

# This file will store: tweet_count,user.screen_name,entities.hastags
csv_file_hash = 'Output_v5_hash.csv'

In [ ]:
#Code extracted from https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console/3173338
#We used beacause helps to visualize
# Prints a progress bar
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
# Functions to read the different files and initialize csv
def read_tweet_data(tweet_count,tweet):
    field_list = ['created_at','favorite_count','retweet_count']
    
    csv_data_string=str(tweet_count)+','
    
    ###
    text=str(tweet['full_text'])
    text=clean(text)
    csv_data_string+=('\"'+text+'\",')
    csv_data_string+=clean(str(tweet['user']['screen_name']))+','
    ##
    for field in field_list:
        csv_data_string+=clean(str(tweet[field]))+','
    csv_data_string+='https://twitter.com/twitter/statuses/'+clean(str(tweet['id']))+'\n'
    
    return (csv_data_string)

        
def read_ment_data(tweet_count,tweet):
    if (len(tweet['entities']['user_mentions'])>0):
        csv_ment_string=''
        for mention in tweet['entities']['user_mentions']:
            csv_ment_string+=(str(tweet_count)+','+str(tweet['user']['screen_name'])+','
                             +str(mention['screen_name'])+'\n')
        return(csv_ment_string)
    return('')

def read_hash_data(tweet_count,tweet):
    if (len(tweet['entities']['hashtags'])>0):
        csv_hash_string=''
        for hashtag in tweet['entities']['hashtags']:
            csv_hash_string+=(str(tweet_count)+','+str(tweet['user']['screen_name'])+','
                             +str(hashtag['text'])+'\n')
        return(csv_hash_string)
    return('')
        
def initialize_csv():
    tweet_data='tweet_count,text,user_screen_name,created_at,favorite_count,retweet_count,url\n'   
    tweet_ment='tweet_count,user_mentioner,user_mentionated\n'
    tweet_hash='tweet_count,user_screen_name,hastags\n'
    return(tweet_data,tweet_ment,tweet_hash)


In [ ]:
# Functions to read and write blocks of tweets (with all the information needed) to the corresponding files
def read_next_block(f_jsonfile,window,pointer):
    block=[]
    block.append(f_jsonfile.readline())
    if(block[0]!=''): 
        for i in range(window-1):
            block.append(f_jsonfile.readline())
        pointer.append(block)

def write_data(tweet_data,tweet_ment,tweet_hash):
    f_data=open(csv_file_data, 'a+',encoding="utf-8",errors='ignore')
    f_data.write(tweet_data)
    f_data.close()
    f_ment=open(csv_file_ment, 'a+',encoding="utf-8",errors='ignore')
    f_ment.write(tweet_ment)
    f_ment.close()
    f_hash=open(csv_file_hash, 'a+',encoding="utf-8",errors='ignore')
    f_hash.write(tweet_hash)
    f_hash.close()

    
def read_block(data,block,dump):
    tweet_count = data.tweet_count()
    
    if tweet_count==0:
        tweet_data,tweet_ment,tweet_hash=initialize_csv()
    else:
        tweet_data=''
        tweet_ment=''
        tweet_hash=''
    for tweet in block:
        try:
            tweet=json.loads(tweet)
            if ((tweet_count<data.end())&(tweet['lang']=='en')):
                
                tweet_data_temp=read_tweet_data(tweet_count,tweet)

                tweet_ment_temp=read_ment_data(tweet_count,tweet)
                tweet_hash_temp=read_hash_data(tweet_count,tweet)

                #if doesnt break append
                tweet_data+=tweet_data_temp
                tweet_ment+=tweet_ment_temp
                tweet_hash+=tweet_hash_temp
                tweet_count+=1
        except Exception as e:
            
            dump[0]+=1
            #dump[1]+=(str(tweet)+'\n')    
            
    write_data(tweet_data,tweet_ment,tweet_hash)
    data.update(tweet_count)

def clean(string):
    string=re.sub(r'\"','', string)
    string=re.sub(r',','', string)
    string=re.sub(r'\n','', string)
    return(string)


In [ ]:
#from threading import Thread
#BREAK#This is to avoid accidental runs that may affect your data
import time

start_time=time.time()
data = control(properties_data)
with open(json_file_corrected, 'r') as f_jsonfile:
    old= data.tweet_count()
    dump=[0,'']
    while(data.tweet_count()<data.end()):
        old= data.tweet_count()
        block=[]
        dump[1]=[]
        read_next_block(f_jsonfile,data.window(),block)
        read_block(data,block[0],dump)
        if len(block)==0:break

        dtime=time.time()-start_time
        printProgressBar((data.start()+data.tweet_count()),data.end(),length=75,
                             suffix = 'Complete, time: '+str(dtime)+ ', estimation {}'.format(((data.end()-data.start())/(data.start()+data.tweet_count())*dtime/60)))
        data.save()
    

 |███████████████████████████████████████████████████████████████████████████| 100.0% Complete, time: 203.94645762443542, estimation 3.3991076270739238
